In [1]:
import pandas as pd

from tabular.pipeline import *
from tabular.experiments import *

In [2]:
params = {
    "title": "test",
    "notes": " some human readable sentence",
    "experiment": "linear_reg",
    "data_name": "data/train.csv",
    "folds": 5,
    "pipeline": "noop",
    "metrics": ["mse"],
    "x_cols": ["building_id"],
    "y_cols": ["meter_reading"],
}

In [3]:
res = run_experiment(params, "records/")

In [4]:
res